In [ ]:
pip install aiogram

In [ ]:
pip install openai

In [ ]:
import asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
import os
import logging
import sys
import openai

In [ ]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('Openai_api_key')
os.environ['TELEGRAM_BOT_TOKEN'] = userdata.get('TELEGRAM_BOT_TOKEN')

In [ ]:
dp = Dispatcher()

In [ ]:
class Gpt_context:
  "This will store previous responses from chatgpt"

  def __init__(self):
    self.response =""

In [ ]:
context = Gpt_context()

In [ ]:
@dp.message(Command('start'))
async def command_start_handler(message: types.Message):
    "This handler will recieve the message with '/start' command"
    await message.answer("hii im a bot made bt rajvansh. \n how can i assist you?")

In [ ]:
@dp.message(Command('help'))
async def command_start_handler(message: types.Message):
    "This handler will recieve the message with '/help' command"
    help_command = """
    HELLO bro , Tension nai leneka apun hai na .. koi ek command type kar -
    /start
    /help"""
    await message.reply(help_command)

In [ ]:
# Add this at the top (after imports)
from collections import defaultdict
import tiktoken  # For token counting

# Initialize conversation history
conversations = defaultdict(list)
MAX_HISTORY_TOKENS = 3000  # Adjust based on your token limits

# Initialize OpenAI client
client = openai.AsyncOpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Add reset command handler
@dp.message(Command('reset'))
async def reset_conversation(message: types.Message):
    chat_id = message.chat.id
    conversations[chat_id] = []  # Clear history
    await message.answer("Conversation history cleared! Let's start fresh.")

# Updated message handler
@dp.message()
async def chatgpt(message: types.Message):
    chat_id = message.chat.id

    # Add user message to history
    conversations[chat_id].append({
        "role": "user",
        "content": message.text
    })

    try:
        # Get response from OpenAI with full history
        response = await client.chat.completions.create(
            model='gpt-4o-mini',
            messages=conversations[chat_id],
            temperature=0.7
        )
        response_content = response.choices[0].message.content

        # Add assistant response to history
        conversations[chat_id].append({
            "role": "assistant",
            "content": response_content
        })

        # Truncate history if too long
        conversations[chat_id] = truncate_history(
            conversations[chat_id],
            MAX_HISTORY_TOKENS
        )

        await message.reply(response_content)

    except Exception as e:
        await message.reply(f"Error: {str(e)}")
        conversations[chat_id].pop()  # Remove failed user message

# Token counting and history truncation
def truncate_history(history, max_tokens):
    encoder = tiktoken.encoding_for_model("gpt-4")
    total_tokens = 0
    new_history = []

    # Process messages in reverse (keep most recent)
    for msg in reversed(history):
        msg_tokens = len(encoder.encode(msg["content"]))
        if total_tokens + msg_tokens > max_tokens:
            break
        new_history.insert(0, msg)  # Add to front
        total_tokens += msg_tokens

    return new_history

In [ ]:
#@dp.message()
#async def repeat(message: types.Message):
 # "Handles all other messages apart from start and help command"

 # await message.reply(message.text)

@dp.message()
async def chatgpt(message: types.Message):
  "This will process the messages and will get response from openai"
  response = openai.chat.completions.create(
      model='gpt-4o-mini',
      messages=[
          {'role': 'user', 'content': message.text}
      ]
  )

  value = response.choices[0].message.content
  print("Bot Response:", value)
  await message.reply(value)

In [ ]:
async def main() -> None:
  bot = Bot(os.environ['TELEGRAM_BOT_TOKEN'])
  await dp.start_polling(bot)

In [ ]:
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO, stream=sys.stdout)
  await main()